In [2]:
def firstStageRegress(variables, endo):
    """First Stage Regression"""
    
    X_stage1 = sm.add_constant(df[variables])
    y_stage1 = df[endo]

    # Fit the regression to find the predicted values
    results_stage1 = sm.OLS(y_stage1, X_stage1).fit()
    predictor = results_stage1.predict(X_stage1)

    return results_stage1, predictor 

def secondStageRegress(variables, predict):  
    """Second Stage Regression"""
    
    X_stage2 = sm.add_constant(df[variables])
    y_stage2 = df[predict]  

    # Fit the regression model
    results_stage2 = sm.OLS(y_stage2, X_stage2).fit()
    
    return results_stage2

In [3]:
# Read dataset
df = pd.read_csv("Data-GP1-1(updated).csv")
df

,Mon,Tue,Wed,Thu,Date,Jan,Feb,Mar,Apr,May,Month,Year,Stormy,Mixed,p,q,Rainy,Cold,Wind
0,1,0,0,0,2,0,0,0,0,0,12,91,1,0,-0.430783,8.994421,1,0,2.995732
1,0,1,0,0,3,0,0,0,0,0,12,91,1,0,0.000000,7.707063,0,0,2.995732
2,0,0,1,0,4,0,0,0,0,0,12,91,0,1,0.072321,8.350194,1,1,2.813411
3,0,0,0,1,5,0,0,0,0,0,12,91,1,0,0.247139,8.656955,0,1,3.036554
4,0,0,0,0,6,0,0,0,0,0,12,91,1,0,0.664327,7.844241,0,1,3.036554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1,0,0,0,4,0,0,0,0,1,5,92,0,0,-0.798508,8.610683,0,0,2.862201
107,0,1,0,0,5,0,0,0,0,1,5,92,0,1,-0.087011,7.162397,0,0,2.908721
108,0,0,1,0,6,0,0,0,0,1,5,92,0,1,0.184922,7.362010,0,0,2.862201
109,0,0,0,1,7,0,0,0,0,1,5,92,0,1,0.223143,8.764053,0,0,2.813411


In [4]:
# Declare Variables that will be the same across all models

Endo = ["p"]
Predict = ["q"]

---

# Model 1 (Exo: Days of the Week) (IV: Stormy, Mixed)

In [5]:
# Declare IVs
IV = ["Stormy", "Mixed"]
Exo = ["Mon", "Tue","Wed","Thu"]

# First Stage Regression
result1, p_hat = firstStageRegress(IV + Exo, Endo)
df["phat"] = p_hat
print(result1.summary(),"\n")

# Second Stage Regression
result2 = secondStageRegress(["phat"] + Exo, Predict)
print(result2.summary(),"\n")

mlr2 = lm.IV2SLS(dependent=df[["q"]], 
                     exog=df[Exo], 
                     endog=df[["p"]], 
                     instruments=df[IV]).fit(cov_type="homoskedastic", debiased=True)
print(mlr2.wu_hausman(),"\n")
print(mlr2.sargan)


                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.245
Model:                            OLS   Adj. R-squared:                  0.201
Method:                 Least Squares   F-statistic:                     5.624
Date:                Thu, 21 Sep 2023   Prob (F-statistic):           4.35e-05
Time:                        19:33:12   Log-Likelihood:                -34.566
No. Observations:                 111   AIC:                             83.13
Df Residuals:                     104   BIC:                             102.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3596      0.079     -4.543      0.0

---

# Model 2 (Exo: Days of the Week) (IV: Wind)

In [6]:
# Declare IVs
IV = ["Wind"]
Exo = ["Mon", "Tue","Wed","Thu"]

# First Stage Regression
result1, p_hat = firstStageRegress(IV + Exo, Endo)
df["phat"] = p_hat
print(result1.summary(),"\n")

# Second Stage Regression
result2 = secondStageRegress(["phat"] + Exo, Predict)
print(result2.summary(),"\n")

mlr2 = lm.IV2SLS(dependent=df[["q"]], 
                     exog=df[Exo], 
                     endog=df[["p"]], 
                     instruments=df[IV]).fit(cov_type="homoskedastic", debiased=True)
print(mlr2.wu_hausman(),"\n")
print(mlr2.sargan)


                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.203
Model:                            OLS   Adj. R-squared:                  0.165
Method:                 Least Squares   F-statistic:                     5.343
Date:                Thu, 21 Sep 2023   Prob (F-statistic):           0.000205
Time:                        19:33:12   Log-Likelihood:                -37.580
No. Observations:                 111   AIC:                             87.16
Df Residuals:                     105   BIC:                             103.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.5084      0.468     -5.358      0.0

---

# Model 3 (Exo: Days of the Week, Rainy, Cold) (IV: Wind)

In [9]:
# Declare IVs
IV = ['Wind']
Exo = ["Mon","Tue","Wed","Thu", "Rainy", "Cold"]

# First Stage Regression
result1, p_hat = firstStageRegress(IV + Exo, Endo)
df["phat"] = p_hat
print(result1.summary(),"\n")

# Second Stage Regression
result2 = secondStageRegress(["phat"] + Exo, Predict)
print(result2.summary(),"\n")

mlr2 = lm.IV2SLS(dependent=df[["q"]], 
                     exog=df[Exo], 
                     endog=df[["p"]], 
                     instruments=df[IV]).fit(cov_type="homoskedastic", debiased=True)
print(mlr2.wu_hausman(),"\n")
print(mlr2.sargan)

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     3.830
Date:                Thu, 21 Sep 2023   Prob (F-statistic):           0.000973
Time:                        19:33:12   Log-Likelihood:                -37.323
No. Observations:                 111   AIC:                             90.65
Df Residuals:                     103   BIC:                             112.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.3696      0.514     -4.611      0.0

---

# Model 4 (Exo: Months of a Year) (IV: Wind)

In [7]:
# Declare IVs
IV = ["Wind"]
Exo = ["Jan","Feb","Mar","Apr","May"]

# First Stage Regression
result1, p_hat = firstStageRegress(IV + Exo, Endo)
df["phat"] = p_hat
print(result1.summary(),"\n")

# Second Stage Regression
result2 = secondStageRegress(["phat"] + Exo, Predict)
print(result2.summary(),"\n")

mlr2 = lm.IV2SLS(dependent=df[["q"]], 
                     exog=df[Exo], 
                     endog=df[["p"]], 
                     instruments=df[IV]).fit(cov_type="homoskedastic", debiased=True)
print(mlr2.wu_hausman(),"\n")
print(mlr2.sargan)

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.227
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     5.103
Date:                Thu, 21 Sep 2023   Prob (F-statistic):           0.000125
Time:                        19:33:12   Log-Likelihood:                -35.840
No. Observations:                 111   AIC:                             85.68
Df Residuals:                     104   BIC:                             104.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8246      0.525     -3.477      0.0

---

# Model 5 (Exo: Months of a Year) (IV: Stormy, Mixed)

In [8]:
# Declare IVs
IV = ['Stormy', 'Mixed']
Exo = ["Jan","Feb","Mar","Apr","May"]

# First Stage Regression
result1, p_hat = firstStageRegress(IV + Exo, Endo)
df["phat"] = p_hat
print(result1.summary(),"\n")

# Second Stage Regression
result2 = secondStageRegress(["phat"] + Exo, Predict)
print(result2.summary(),"\n")

mlr2 = lm.IV2SLS(dependent=df[["q"]], 
                     exog=df[Exo], 
                     endog=df[["p"]], 
                     instruments=df[IV]).fit(cov_type="homoskedastic", debiased=True)
print(mlr2.wu_hausman(),"\n")
print(mlr2.sargan)

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.302
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     6.381
Date:                Thu, 21 Sep 2023   Prob (F-statistic):           3.07e-06
Time:                        19:33:12   Log-Likelihood:                -30.169
No. Observations:                 111   AIC:                             76.34
Df Residuals:                     103   BIC:                             98.01
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2661      0.091     -2.937      0.0

---
# Model 6 (Exo: Months of a Year, Rainy, Cold), (IV: Stormy, Mixed) 

In [10]:
# Declare IVs
IV = ['Stormy', 'Mixed']
Exo = ["Jan","Feb","Mar","Apr","May", "Rainy", "Cold"]

# First Stage Regression
result1, p_hat = firstStageRegress(IV + Exo, Endo)
df["phat"] = p_hat
print(result1.summary(),"\n")

# Second Stage Regression
result2 = secondStageRegress(["phat"] + Exo, Predict)
print(result2.summary(),"\n")

mlr2 = lm.IV2SLS(dependent=df[["q"]], 
                     exog=df[Exo], 
                     endog=df[["p"]], 
                     instruments=df[IV]).fit(cov_type="homoskedastic", debiased=True)
print(mlr2.wu_hausman(),"\n")
print(mlr2.sargan)

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.313
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     5.110
Date:                Thu, 21 Sep 2023   Prob (F-statistic):           1.08e-05
Time:                        19:33:12   Log-Likelihood:                -29.335
No. Observations:                 111   AIC:                             78.67
Df Residuals:                     101   BIC:                             105.8
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3041      0.102     -2.990      0.0

---
# Model 7 (Exo: Months of a Year, Rainy, Cold), (IV: Wind)

In [11]:
# Declare IVs
IV = ['Wind']
Exo = ["Jan","Feb","Mar","Apr","May", "Rainy", "Cold"]

# First Stage Regression
result1, p_hat = firstStageRegress(IV + Exo, Endo)
df["phat"] = p_hat
print(result1.summary(),"\n")

# Second Stage Regression
result2 = secondStageRegress(["phat"] + Exo, Predict)
print(result2.summary(),"\n")

mlr2 = lm.IV2SLS(dependent=df[["q"]], 
                     exog=df[Exo], 
                     endog=df[["p"]], 
                     instruments=df[IV]).fit(cov_type="homoskedastic", debiased=True)
print(mlr2.wu_hausman(),"\n")
print(mlr2.sargan)

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.232
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                     3.862
Date:                Thu, 21 Sep 2023   Prob (F-statistic):           0.000522
Time:                        19:33:12   Log-Likelihood:                -35.477
No. Observations:                 111   AIC:                             88.95
Df Residuals:                     102   BIC:                             113.3
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.7371      0.542     -3.206      0.0

Dun keep Model 7

---
# Model 8 (Exo: Months of a Year, Days of the Week, Rainy, Cold), (IV: Stormy, Mixed)

In [12]:
# Declare IVs
IV = ['Stormy', 'Mixed']
Exo = ["Mon","Tue","Wed","Thu","Jan","Feb","Mar","Apr","May", "Rainy", "Cold"]

# First Stage Regression
result1, p_hat = firstStageRegress(IV + Exo, Endo)
df["phat"] = p_hat
print(result1.summary(),"\n")

# Second Stage Regression
result2 = secondStageRegress(["phat"] + Exo, Predict)
print(result2.summary(),"\n")

mlr2 = lm.IV2SLS(dependent=df[["q"]], 
                     exog=df[Exo], 
                     endog=df[["p"]], 
                     instruments=df[IV]).fit(cov_type="homoskedastic", debiased=True)
print(mlr2.wu_hausman(),"\n")
print(mlr2.sargan)

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.334
Model:                            OLS   Adj. R-squared:                  0.244
Method:                 Least Squares   F-statistic:                     3.738
Date:                Thu, 21 Sep 2023   Prob (F-statistic):           7.84e-05
Time:                        19:33:13   Log-Likelihood:                -27.623
No. Observations:                 111   AIC:                             83.25
Df Residuals:                      97   BIC:                             121.2
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2558      0.118     -2.177      0.0

---
# Model 9 (Exo: Months of a Year, Days of the Week, Rainy, Cold), (IV: Wind)

In [13]:
# Declare IVs
IV = ['Wind']
Exo = ["Mon","Tue","Wed","Thu","Jan","Feb","Mar","Apr","May", "Rainy", "Cold"]

# First Stage Regression
result1, p_hat = firstStageRegress(IV + Exo, Endo)
df["phat"] = p_hat
print(result1.summary(),"\n")

# Second Stage Regression
result2 = secondStageRegress(["phat"] + Exo, Predict)
print(result2.summary(),"\n")

mlr2 = lm.IV2SLS(dependent=df[["q"]], 
                     exog=df[Exo], 
                     endog=df[["p"]], 
                     instruments=df[IV]).fit(cov_type="homoskedastic", debiased=True)
print(mlr2.wu_hausman(),"\n")
print(mlr2.sargan)

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.180
Method:                 Least Squares   F-statistic:                     3.014
Date:                Thu, 21 Sep 2023   Prob (F-statistic):            0.00124
Time:                        19:33:13   Log-Likelihood:                -32.727
No. Observations:                 111   AIC:                             91.45
Df Residuals:                      98   BIC:                             126.7
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9547      0.550     -3.554      0.0

---
&nbsp;
# Model 10 (Exo: Months of a Year, Days of the Week), (IV: Stormy, Mixed)

In [14]:
# Declare IVs
IV = ['Stormy', "Mixed"]
Exo = ["Mon","Tue","Wed","Thu","Jan","Feb","Mar","Apr","May"]

# First Stage Regression
result1, p_hat = firstStageRegress(IV + Exo, Endo)
df["phat"] = p_hat
print(result1.summary(),"\n")

# Second Stage Regression
result2 = secondStageRegress(["phat"] + Exo, Predict)
print(result2.summary(),"\n")

mlr2 = lm.IV2SLS(dependent=df[["q"]], 
                     exog=df[Exo], 
                     endog=df[["p"]], 
                     instruments=df[IV]).fit(cov_type="homoskedastic", debiased=True)
print(mlr2.wu_hausman(),"\n")
print(mlr2.sargan)

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.321
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     4.254
Date:                Thu, 21 Sep 2023   Prob (F-statistic):           3.60e-05
Time:                        19:33:14   Log-Likelihood:                -28.680
No. Observations:                 111   AIC:                             81.36
Df Residuals:                      99   BIC:                             113.9
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2299      0.109     -2.118      0.0

---
&nbsp;
# Model 11 (Exo: Months of a Year, Days of the Week), (IV: Wind)

In [15]:
# Declare IVs
IV = ['Wind']
Exo = ["Mon","Tue","Wed","Thu","Jan","Feb","Mar","Apr","May"]

# First Stage Regression
result1, p_hat = firstStageRegress(IV + Exo, Endo)
df["phat"] = p_hat
print(result1.summary(),"\n")

# Second Stage Regression
result2 = secondStageRegress(["phat"] + Exo, Predict)
print(result2.summary(),"\n")

mlr2 = lm.IV2SLS(dependent=df[["q"]], 
                     exog=df[Exo], 
                     endog=df[["p"]], 
                     instruments=df[IV]).fit(cov_type="homoskedastic", debiased=True)
print(mlr2.wu_hausman(),"\n")
print(mlr2.sargan)

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     3.588
Date:                Thu, 21 Sep 2023   Prob (F-statistic):           0.000425
Time:                        19:33:15   Log-Likelihood:                -33.146
No. Observations:                 111   AIC:                             88.29
Df Residuals:                     100   BIC:                             118.1
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0364      0.532     -3.830      0.0